# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
import joblib
from azureml.core import Environment
from azureml.core.model import InferenceConfig,Model
from azureml.core.webservice import AciWebservice,Webservice,LocalWebservice
from azureml.train.automl import AutoMLConfig
from azureml.core.compute_target import ComputeTargetException
import requests
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math

## Dataset

### Overview
The uploaded dataset is a bond data. The targeted variable is OAS - Option Adjusted Spread. This spread is a quantitative representation of the risk of the bond. It depends on many factors, one of the most important of which is the bonds's rating. The variables representing the rating is Warf (numerical format). Additionally, the dataset contains various indicators such as if the bond belongs to a certain type of industry. The goal is to predict the OAS using the input variables. Since it is a numeric continious variable, a regression will be chosen.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

In [3]:
df_bonds = ws.datasets['df_bonds'] 
experiment=Experiment(ws, 'bonds_oas')
cluster_name='valeriy-cluster'
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here
automl_settings = {
"experiment_timeout_minutes": 15,
"max_concurrent_iterations":20,
'n_cross_validations':2,
"primary_metric" : 'r2_score'
}
project_folder = './bonds-project'
automl_config = AutoMLConfig(compute_target=cluster,
                             task = "regression",
                             training_data=df_bonds,
                             label_column_name="OAS",   
                             path = project_folder,
                             debug_log = "automl_errors.log",                            
                             **automl_settings
                            )

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
remote_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
bonds_oas,AutoML_0a0716a7-ef04-4b1c-bd8f-66b76a8bcec5,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_0a0716a7-ef04-4b1c-bd8f-66b76a8bcec5',
 'target': 'valeriy-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-22T07:19:54.670339Z',
 'endTimeUtc': '2022-02-22T07:44:55.979469Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 15 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'valeriy-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"bonds_oas","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group":"aml-quickstarts-187275","workspace_name":"quick-starts-ws-187275","region":"westeurope","compute_target":"valeriy-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [17]:
best_run_automl, fitted_model_automl=remote_run.get_output()
model_name=best_run_automl.properties['model_name']
print(best_run_automl.properties,model_name)

{'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'regression\',\'primary_metric\':\'r2_score\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'bonds_oas\',\'compute_target\':\'valeriy-cluster\',\'subscription_id\':\'3d1a56d2-7c81-4118-9790-f85d1acf0c77\',\'region\':\'westeurope\',\'spark_service\':None}","ensemble_run_id":"AutoML_0a0716a7-ef04-4b1c-bd8f-66b76a8bcec5_50","experiment_name":"bonds_oas","workspace_name":"quick-starts-ws-187275","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group_name":"aml-quickstarts-187275"}}]}', 'training_percent': '100', 'predicted_cost': None, 'iteration': '50', '_aml_system_scenario_identification': 'Remote.Child', '_azureml.ComputeTargetType': 'amlcompute', 'C

In [14]:
#TODO: Save the best model
joblib.dump(fitted_model_automl, 'best_fitted_model_autolm.pkl')

['best_fitted_model_autolm.pkl']

In [15]:
#Check the performance of the best model. This is required in order to compare it with the hyperdrive one
df_bonds = ws.datasets['df_bonds']
x=df_bonds.to_pandas_dataframe().drop('OAS',1)
y=df_bonds.to_pandas_dataframe()['OAS']
   
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=10)
y_pred_automl = fitted_model_automl.predict(x_test)
rmse_automl = math.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred_automl))
print('automl:',rmse_automl)

automl: 271.9051570541046


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [18]:
#Register model
description = 'Bonds data'
model = remote_run.register_model(model_name=model_name,
                          description=description,
                           tags=None)
print(remote_run.model_id)

AutoML0a0716a7e50


In [19]:
#Create an inference configuration:
env = Environment(name="valeriy_env")
from azureml.core.conda_dependencies import CondaDependencies
script_file_name='score_bonds.py'
best_run_automl.download_file('outputs/scoring_file_v_1_0_0.py',script_file_name)
inference_config = InferenceConfig(entry_script=script_file_name,environment=best_run_automl.get_environment())
aciconfig=AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=1,tags={'type':'bonds_df'}, description='n')
aci_service=Model.deploy(ws,'valeriywebservice7',[model],inference_config,aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-02-22 07:50:03+00:00 Creating Container Registry if not exists..
2022-02-22 08:00:03+00:00 Registering the environment.
2022-02-22 08:00:04+00:00 Generating deployment configuration..
2022-02-22 08:00:09+00:00 Submitting deployment to compute..
2022-02-22 08:00:21+00:00 Checking the status of deployment valeriywebservice7..
2022-02-22 08:05:25+00:00 Checking the status of inference endpoint valeriywebservice7.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [27]:
import json
data_to_predict=x_test.iloc[:2]
test_sample=json.dumps({'data':data_to_predict.to_dict(orient='records')})
response = aci_service.run(input_data=test_sample)
print(response)

{"result": [337.63845133144844, 699.4955018449756]}


TODO: In the cell below, print the logs of the web service and delete the service

In [23]:
#Save the enviroment details
env_used=best_run_automl.get_environment()
env_used.save_to_directory('automl_env.txt', overwrite=True)

In [28]:
print(aci_service.get_logs())
aci_service.delete()

2022-02-22T08:05:18,346878700+00:00 - gunicorn/run 
2022-02-22T08:05:18,345033800+00:00 - rsyslog/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-02-22T08:05:18,356067500+00:00 - iot-server/run 
2022-02-22T08:05:18,387588400+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_6797cf9b513e59b405ce80f3e9222a7d/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-02-22T08:05:18,741841100+00:00 - iot-server/finish 1 0
2022-02-22T08:05:18,749497700+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (70)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 98
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2022-02-22 08:05:22,413 | root | INFO | Starting up app insights client
logging socket was 

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
